<a href="https://colab.research.google.com/github/punamvekariya68/COMP6721_Project/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from PIL import Image, ImageOps, ImageEnhance
from skimage.feature import hog, graycomatrix, graycoprops
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
#Step 1: Load Dataset
base_dir = "Training_Resize"
categories = ['library', 'museum', 'shopping_mall']
img_size = (64, 64)

X = []
y = []

for label, category in enumerate(categories):
    folder = os.path.join(base_dir, category)
    for filename in os.listdir(folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert("RGB").resize(img_size)
            X.append(np.array(img))
            y.append(label)

X = np.array(X)
y = np.array(y)
print(f"Loaded {len(X)} total images.")

In [ ]:
# Step 2 - Feature Extraction
print("Extracting features...")

def extract_hog_features(images):
    return np.array([
        hog(rgb2gray(img), pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        for img in images
    ])

def extract_color_histogram(image, bins=32):
    chans = image.transpose(2, 0, 1)
    hist = [np.histogram(ch, bins=bins, range=(0, 256))[0] for ch in chans]
    return np.concatenate(hist)

def extract_glcm_features(img):
    gray = rgb2gray(img)
    gray = (gray * 255).astype(np.uint8)
    glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    return [graycoprops(glcm, p)[0, 0] for p in props]

hog_features = extract_hog_features(X)
color_features = np.array([extract_color_histogram(img) for img in X])
glcm_features = np.array([extract_glcm_features(img) for img in X])

X_combined = np.hstack([hog_features, color_features, glcm_features])
print(f"Combined feature shape: {X_combined.shape}")